In [ ]:
import os

import cv2
import pandas as pd
import numpy as np

from skimage.feature import local_binary_pattern

Paths are initialized

In [ ]:
#wikiart (large dataset) will be added later

kaggle_train_path = "../data/train"
kaggle_test_path = "../data/test"

features_path = "../features"
painters_path = "../data/artists.csv"

kaggle_painter_names = os.listdir(kaggle_train_path)
painters = pd.read_csv(painters_path)
painters = painters["name"]

Feature 1: Average of HOG

In [ ]:
#Computing HOG over an image
def compute_hog(image):
    hog_desc = cv2.HOGDescriptor()
    hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    hog_features = []

    for channel in range(hsv_image.shape[2]): #computing for HSV channels
        hog_features_channel = hog_desc.compute(hsv_image[:, :, channel])
        hog_features.append(hog_features_channel)

    return np.concatenate(hog_features)

#Computing average of HOG for each painter's paintings distinctively
for painter in kaggle_painter_names:
    paintings_path = os.path.join(kaggle_train_path, painter)
    images_list = [image for image in os.listdir(paintings_path)]

    hog_features_list = []
    for image_name in images_list:
        image_path = os.path.join(paintings_path, image_name)
        image = cv2.imread(image_path)
        hog_features = compute_hog(image)
        hog_features_list.append(hog_features)

    hog_features_array = np.array(hog_features_list)
    avg_hog_features = np.mean(hog_features_array, axis=0)

Feature 2: Average of LBP

In [ ]:
def compute_lbp(image):
    hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    gray_image = hsv_image[:, :, 2] #to obtain texture information

    radius = 3
    n_points = 8 * radius
    features = local_binary_pattern(gray_image, n_points, radius, method="uniform")
    lbp_hist, _ = np.histogram(features.ravel(), bins=np.arange(0, n_points + 3), range=(0, n_points + 2))
    lbp_hist /= np.sum(lbp_hist)
    return lbp_hist

average_lbp_features = {}

#Computing average of LBP for each painter's paintings distinctively
for painter in kaggle_painter_names:
    paintings_path = os.path.join(kaggle_train_path, painter)
    images_list = [image for image in os.listdir(paintings_path)]
    painter_lbp_features = []

    for image_name in images_list:
        image_path = os.path.join(paintings_path, image_name)
        image = cv2.imread(image_path)
        lbp_feature = compute_lbp(image)
        painter_lbp_features.append(lbp_feature)

    average_lbp_features[painter] = np.mean(painter_lbp_features, axis=0)

Feature 3: Standard Deviation of HOG

In [ ]:
#Computing standard deviation of HOG for each painter's paintings distinctively
for painter in kaggle_painter_names:
    paintings_path = os.path.join(kaggle_train_path, painter)
    images_list = [image for image in os.listdir(paintings_path)]

    hog_features_list = []
    for image_name in images_list:
        image_path = os.path.join(paintings_path, image_name)
        image = cv2.imread(image_path)
        hog_features = compute_hog(image)
        hog_features_list.append(hog_features)

    hog_features_array = np.array(hog_features_list)
    avg_hog_features = np.std(hog_features_array, axis=0)

Feature 4: Standard Deviation of LBP

In [ ]:
#Computing standard deviation of LBP for each painter's paintings distinctively
for painter in kaggle_painter_names:
    paintings_path = os.path.join(kaggle_train_path, painter)
    images_list = [image for image in os.listdir(paintings_path)]
    painter_lbp_features = []

    for image_name in images_list:
        image_path = os.path.join(paintings_path, image_name)
        image = cv2.imread(image_path)
        lbp_feature = compute_lbp(image)
        painter_lbp_features.append(lbp_feature)

    average_lbp_features[painter] = np.std(painter_lbp_features, axis=0)

Feature 5: Average of Gray Histogram

In [ ]:
gray_hist_avg_path = "../features/gray_hist_avg.txt"

def compute_gray_histogram(image):
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    hist, _ = np.histogram(gray_image.ravel(), bins=256, range=[0,256])
    return hist

avg_gray_hist = {}

with open(gray_hist_avg_path, "w") as gray_hist_avg:
    for painter in kaggle_painter_names:
        gray_hist_avg.write(painter)
        gray_hist_avg.write("\n")

        paintings_path = os.path.join(kaggle_train_path, painter)
        painter_gray_hist = []

        for image_name in os.listdir(paintings_path):
            image_path = os.path.join(paintings_path, image_name)
            image = cv2.imread(image_path)
            gray_hist = compute_gray_histogram(image)
            painter_gray_hist.append(gray_hist)

        avg_gray_hist[painter] = np.mean(painter_gray_hist, axis=0)
        np.savetxt(gray_hist_avg, [avg_gray_hist[painter]])

Feature 6: Average of Image Edges Histogram

In [ ]:
edges_hist_avg_path = "../features/edges_hist_avg.txt"

def compute_edge_histogram(image):
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    edges = cv2.Canny(gray_image, 50, 150)
    hist, _ = np.histogram(edges.ravel(), bins=256, range=[0,256])
    return hist

avg_edge_hist = {}

with open(edges_hist_avg_path, "w") as edges_hist_avg:
    for painter in kaggle_painter_names:
        edges_hist_avg.write(painter)
        edges_hist_avg.write("\n")

        paintings_path = os.path.join(kaggle_train_path, painter)
        painter_edge_hist = []

        for image_name in os.listdir(paintings_path):
            image_path = os.path.join(paintings_path, image_name)
            image = cv2.imread(image_path)
            edge_hist = compute_edge_histogram(image)
            painter_edge_hist.append(edge_hist)

        avg_edge_hist[painter] = np.mean(painter_edge_hist, axis=0)
        np.savetxt(edges_hist_avg, [avg_edge_hist[painter]], fmt="%.6f")

Feature 7: Means of Image Edges Histogram

In [ ]:
edges_feat_mean_path = "../features/edges_feat_mean.txt"

def compute_edge_mean(image):
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    edges = cv2.Canny(gray_image, 50, 150)
    edge_mean = np.mean(edges)
    return edge_mean

edge_features_means = {}

with open(edges_feat_mean_path, "w") as edges_feat_mean:
    for painter in kaggle_painter_names:
        edges_feat_mean.write(painter)
        edges_feat_mean.write("\n")

        paintings_path = os.path.join(kaggle_train_path, painter)
        painter_edge_means = []

        for image_name in os.listdir(paintings_path):
            image_path = os.path.join(paintings_path, image_name)
            image = cv2.imread(image_path)
            edge_mean = compute_edge_mean(image)
            painter_edge_means.append(edge_mean)

        edge_features_means[painter] = np.mean(painter_edge_means)
        np.savetxt(edges_feat_mean, [edge_features_means[painter]], fmt="%.6f")

Feature 8: Average of Histogram for S Component From HSV Image

In [ ]:
s_hsv_avg_path = "../features/s_hsv_avg.txt"

def compute_s_histogram(image):
    hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    s_component = hsv_image[:,:,1]
    hist, _ = np.histogram(s_component.ravel(), bins=256, range=[0,256])
    return hist

avg_s_hist = {}

with open(s_hsv_avg_path, "w") as s_hsv_avg:
    for painter in kaggle_painter_names:
        s_hsv_avg.write(painter)
        s_hsv_avg.write("\n")

        paintings_path = os.path.join(kaggle_train_path, painter)
        painter_s_hist = []
        
        for image_name in os.listdir(paintings_path):
            image_path = os.path.join(paintings_path, image_name)
            image = cv2.imread(image_path)
            s_hist = compute_s_histogram(image)
            painter_s_hist.append(s_hist)
            
        avg_s_hist[painter] = np.mean(painter_s_hist, axis=0)
        np.savetxt(s_hsv_avg, [avg_s_hist[painter]], fmt="%.6f")

Feature 9: Standard Deviation of Histogram for S Component From HSV Image

In [ ]:
s_hsv_std_path = "../features/s_hsv_std.txt"

std_s_hist = {}

with open(s_hsv_std_path, "w") as s_hsv_std:
    for painter in kaggle_painter_names:
        s_hsv_std.write(painter)
        s_hsv_std.write("\n")

        paintings_path = os.path.join(kaggle_train_path, painter)
        painter_s_hist = []
        
        for image_name in os.listdir(paintings_path):
            image_path = os.path.join(paintings_path, image_name)
            image = cv2.imread(image_path)
            s_hist = compute_s_histogram(image)
            painter_s_hist.append(s_hist)
            
        std_s_hist[painter] = np.std(painter_s_hist, axis=0)
        np.savetxt(s_hsv_std, [std_s_hist[painter]], fmt="%.6f")

Feature 10: Average of Histogram for B Component From RGB Image

In [ ]:
def compute_b_histogram(image):
    b, g, r = cv2.split(image)
    hist, _ = np.histogram(b.ravel(), bins=256, range=[0,256])
    return hist

avg_b_hist = {}

for painter in kaggle_painter_names:
    paintings_path = os.path.join(kaggle_train_path, painter)
    painter_b_hist = []
    
    for image_name in os.listdir(paintings_path):
        image_path = os.path.join(paintings_path, image_name)
        image = cv2.imread(image_path)
        b_hist = compute_b_histogram(image)
        painter_b_hist.append(s_hist)
        
    avg_b_hist[painter] = np.mean(painter_b_hist, axis=0)

Feature 11: Standard Deviation of Histogram for B Component From RGB Image

In [ ]:
std_b_hist = {}

for painter in kaggle_painter_names:
    paintings_path = os.path.join(kaggle_train_path, painter)
    painter_b_hist = []
    
    for image_name in os.listdir(paintings_path):
        image_path = os.path.join(paintings_path, image_name)
        image = cv2.imread(image_path)
        b_hist = compute_b_histogram(image)
        painter_b_hist.append(s_hist)
        
    std_b_hist[painter] = np.std(painter_b_hist, axis=0)